## Aqui fazemos a primeira etapa do code, onde é feito o processamento dos dados baixados da ação correspondente, direto do site da Fundamentus. São explicados todos os passos do processamento. 

In [ ]:
import numpy as np
import pandas as pd
import pandas_datareader.data as web
from datetime import datetime
import yfinance as yf

pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# DADOS FUNDAMENTALISTAS - FUNDAMENTUS

Ao fazer o download do site da Fundamentus, os dados vem em formato excel, então pegamos e convertemos facilmente para cvs.

Aqui trazemos o csv da ação para o notebook. Ele necessita do ".T" para que ele faça uma transposição, pois os atributos estavam nas linhas e as amostras nas colunas, assim invertemos isso, ficando no formato convencional utilizado.

Após, dropamos uma coluna inútil.

A coluna que mostra as datas, que estão separadas por trimestre, fica com um título NaN e está na primeira posição das colunas, assim precisamos atribuir o devido nome a coluna.

Assim, criamos uma coluna chamada "Data Trimeste", ao que lhe é atribuida as datas, e na sequência deletamos a coluna com o títlo NaN, através do iloc que engloba apenas colunas que tenham título "notnull"(naõ nulos).


A coluna "Data Trimestre" fica posicionada como última coluna do dataset, para ficar mais claro, apagamos, colocamos em uma variavel auxiliar, e a inserimos na primeira posição das colunas.

In [ ]:
# COMEÇA O CÓDIGO QUE EXTRAI OS DADOS FUNDAMENTALISTAS

fundamental_data = pd.read_csv("natu3.csv",index_col=0, header=None).T
fundamental_data.drop(columns = 'XLSWrite 1.34 Copyright(c) 1999,2000 Axolot Data', inplace = True)


fundamental_data['Data Trimestre'] = fundamental_data.iloc[0:,0]
fundamental_data = fundamental_data.iloc[:, fundamental_data.columns.notnull()]

data = fundamental_data['Data Trimestre']
fundamental_data.drop(labels=['Data Trimestre'], axis=1,inplace = True)
fundamental_data.insert(0, 'Data Trimestre', data)

fundamental_data.head()

Aqui é o processo de manipulação das datas, muito importante na execução do code.

Primeiro utilizamos pd.to_datetime, função própria do Pandas para que ele entenda o formato e de que se trata de informações sobre data.

Subdividimos em colunas distintas, informações sobre "Dia", "Mês" e "Ano", logo a coluna "Data Trimestre" já não nos era mais útil.

Assim, novamente colocamos as três novas colunas em váriaveis auxiliares, para posiciona-las nas primeiras colunas do dataset.

In [ ]:
fundamental_data['Data Trimestre'] = pd.to_datetime(fundamental_data['Data Trimestre'].str.strip(), format='%d/%m/%Y')


fundamental_data["Day"] = fundamental_data['Data Trimestre'].map(lambda x: x.day)
fundamental_data["Month"] = fundamental_data['Data Trimestre'].map(lambda x: x.month)
fundamental_data["Year"] = fundamental_data['Data Trimestre'].map(lambda x: x.year)

fundamental_data.drop(columns = 'Data Trimestre', inplace = True)

year = fundamental_data['Year']
fundamental_data.drop(labels=['Year'], axis=1,inplace = True)
fundamental_data.insert(0, 'Year', year)

month = fundamental_data['Month']
fundamental_data.drop(labels=['Month'], axis=1,inplace = True)
fundamental_data.insert(1, 'Month', month)

day = fundamental_data['Day']
fundamental_data.drop(labels=['Day'], axis=1,inplace = True)
fundamental_data.insert(2, 'Day', day)

fundamental_data.head()

Essa é uma parte bem crítica, resetamos o índice(para que ele fique com a contagem correta de 0 até o total de linhas). E você deve estar se perguntando: por quê pegar o dataset a partir da 4 posição ?

Esse dataset está em formato descrescente com base nas datas, então, la no final do projeto, quando formos comparar os algortimos de classificação com a série temporal, para validação temos que comparar com dados que já ocorreram, par ter uma noção do desempenho.

Lembrando que se pegassemos de 0:, nesse caso, ele ia predizer dados a partir do 3T de 2018 até o 3T de 2019, sendo que o TCC estava sendo feito nessa época, então não tinhamos os dados reais necessarios para comparação, então por fins logisticos, reduzimos a ánalise um ano antes, para ao final do trabalho, quando compararmos os fundamentus e os algortimos de classificação com as séries temporais e a LSTM tenhamos tranquilidade para comparar com dados que ocorreram de fato para se ter uma boa base para inferir desempenho e tirar conclusões.

In [ ]:
fundamental_data.reset_index(drop=True, inplace=True)
fundamental_data = fundamental_data[4:28]
fundamental_data.reset_index(drop=True, inplace=True)

fundamental_data.head()

Para fins didáticos, criei uma coluna "Quarter" que indica qual trimestre corresponde aquela linha, e a posicionei na primeira posição das colunas.

In [ ]:
fundamental_data['Quarter'] = '2Q2018', '1Q2018', '4Q2017', '3Q2017', '2Q2017', '1Q2017', '4Q2016', '3Q2016', '2Q2016', '1Q2016', '4Q2015', '3Q2015', '2Q2015', '1Q2015','4Q2014', '3Q2014', '2Q2014', '1Q2014','4Q2013', '3Q2013', '2Q2013', '1Q2013', '4Q2012', '3Q2012'

quarter = fundamental_data['Quarter']
fundamental_data.drop(labels=['Quarter'], axis=1, inplace=True)
fundamental_data.insert(0, 'Quarter', quarter)

fundamental_data.head()

Renomeei a coluna "Month" para "Cluster_Month", pois como se trata de trimestre, ele agrupa ali três meses correspondentes áquele trimestre, então sabemos que colunas onde tem Cluster_Month 3, englona na verdade, os meses 1,2 e 3. E assim por diante com os demais.

A coluna dia não nos é mais útil aqui, então tiramos a coluna.

In [ ]:
fundamental_data.rename(columns={'Month' : 'Cluster_Month'},inplace = True)
fundamental_data.drop('Day', axis=1, inplace=True)

fundamental_data.head()

Correção de processamento, os dados vieram com "pontos" o que inviabiliza a manipulação númerica que o pandas compreende, assim tivemos que remover os pontos e atribuir o formato float, anteriormente veio com formato string.

Assim podemos ver a disposição dos números em relação a tabela gerada pela célula abaixo e a que tinhamos antes.

In [ ]:
for key in ['Ativo Total','Ativo Circulante','Caixa e Equivalentes de Caixa AC','Aplicações Financeiras AC','Contas a Receber AC','Estoques AC', 'Ativos Biológicos AC','Tributos a Recuperar AC','Despesas Antecipadas AC','Outros Ativos Circulantes AC','Ativo Realizavel Longo Prazo ','Aplicações Financeiras Avaliadas a Valor Justo ACLP','Aplicações Financeiras Avaliadas ao Custo Amortizado ACLP','Contas a Receber ACLP','Estoques ACLP','Ativos Biológicos ACLP','Tributos Diferidos ACLP','Despesas Antecipadas ACLP','Créditos com Partes Relacionadas ACLP','Outros Ativos Não Circulantes ACLP','Investimentos','Imobilizado','Intangível','Diferido','Passivo Total','Passivo Circulante','Obrigações Sociais e Trabalhistas PC','Fornecedores PC','Obrigações Fiscais PC','Empréstimos e Financiamentos PC','Passivos com Partes Relacionadas PC','Dividendos e JCP a Pagar PC','Outros PC','Provisões PC', 'Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PC', 'Passivo Não Circulante','Empréstimos e Financiamentos PNC','Passivos com Partes Relacionadas PNC','Outros PNC','Tributos Diferidos PNC','Adiantamento para Futuro Aumento Capital PNC','Provisões PNC','Passivos sobre Ativos Não-Correntes a Venda e Descontinuados PNC','Lucros e Receitas a Apropriar PNC','Participação dos Acionistas Não Controladores','Patrimônio Líquido','Capital Social Realizado','Reservas de Capital','Reservas de Reavaliação','Reservas de Lucros','Lucros/Prejuízos Acumulados','Ajustes de Avaliação Patrimonial','Ajustes Acumulados de Conversão','Outros Resultados Abrangentes','Adiantamento para Futuro Aumento Capital']:  
    fundamental_data[key] = fundamental_data[key].map(lambda x: str(x).replace(".", ""))
    fundamental_data[key] = fundamental_data[key].astype(float)
    
fundamental_data.head()

# DADOS DE PREÇO - API YAHOO FINANCE

Após determinandos os dados fundamentalistas da ação, chegou a hora de extrairmos os dados de preço da API aberta da Yahoo Finance.

Lembrando que temos que convertê-las ao formato de escala tempora de trimestre, para que possamos concatêna-las aos dados fundamentalistas, formando um dataset único da ação. Dados de preço e dados fundamentalistas referentes ao mesmo espaço temporal para cada amostra.

Abaixo, primeiro atribuimos a data de inicio e de fim que coletaremos os dados de preço, e depois trazemos o dataset com base nas datas, e 'm' nos diz respeito a escala temporal mês, como não há trimestre, tivemos que fazer a seguir a manipulação no código, com base nos dados originalmente mensais vindo da API.

In [ ]:
start = datetime(2012, 6, 30)
end = datetime(2018, 5, 31)
price_data = web.get_data_yahoo('NATU3.SA', start, end, interval = 'm')

price_data.head()

As datas na coluna "Date" estavam como índíce, e usamos o code abaixo para tornar "Date" uma coluna normal do dataset, já que vamos precisar manipula-la.

In [ ]:
price_data["Date"] = price_data.index
price_data.reset_index(drop=True, inplace=True)


date = price_data['Date']
price_data.drop(labels=['Date'], axis=1,inplace = True)
price_data.insert(0, 'Date', date)

price_data.head()

Aqui começa a manipulação para transformarmos os dados de preço de escala mensal para escala trimestral.

Tal qual nos dados fundamentalistas, da coluna Date, criamos as colunas referentes a "dia", "mês" e "ano". Assim a coluna Date não nos é mais útil.

Manipulamos para colocar as três colunas referentes a data nas três primeiras posições das colunas no dataset.

In [ ]:
price_data["Day"] = price_data['Date'].map(lambda x: x.day)
price_data["Month"] = price_data['Date'].map(lambda x: x.month)
price_data["Year"] = price_data['Date'].map(lambda x: x.year)

price_data.drop(columns = 'Date', inplace = True)



year = price_data['Year']
price_data.drop(labels=['Year'], axis=1,inplace = True)
price_data.insert(0, 'Year', year)

month = price_data['Month']
price_data.drop(labels=['Month'], axis=1,inplace = True)
price_data.insert(1, 'Month', month)

day = price_data['Day']
price_data.drop(labels=['Day'], axis=1,inplace = True)
price_data.insert(2, 'Day', day)

price_data.head()

Nessa parte, o "for" percorre todas as linhas do dataset, e tal qual no padrão dos dados fundamentalistas, temos que atribuir aquele padrão que chamei de "Cluster_Month", onde o mês 3 representa o trimestre dos meses 1,2 e 3. Mês 6 o trimestre dos meses 4,5 e 6. E assim por diante.

Após, deletamos a coluna "dia", que não nos é útil.

In [ ]:
for i,row in price_data.iterrows():   
    if (row['Month'] == 1) | (row['Month'] == 2) | (row['Month'] == 3):
        price_data.loc[i,'Month'] = 3
    
    elif (row['Month'] == 4) | (row['Month'] == 5) | (row['Month'] == 6):
         price_data.loc[i,'Month'] = 6
        
    elif (row['Month'] == 7) | (row['Month'] == 8) | (row['Month'] == 9):
         price_data.loc[i,'Month'] = 9
        
    else: 
          price_data.loc[i,'Month'] = 12



price_data.drop('Day', axis=1, inplace=True)

price_data.head()

E aqui tempos uma parte bem interessante. Agora agrupamos cada uma das três linhas correspondentes ao trimestre do ano correspondente.

Podemos olhar juntos para a primeira instrução abaixo e para a tabela acima para entender. Agrupamos por Ano e Mês, assim podemos ver as três primeiras linhas com "Month" igual a 3(na verdade são 1,2 e 3) que foram manipulados para facilitar esse processo. Então iremos agrupar essas três linhas, transformando em uma linha só correspondente ao primeiro trimestre de 2014 nesse caso. E assim convertemos os parâmetros de preço conforme suas especificidades.

- Abertura(Open): Utilização do método “first”, atribuindo o primeiro valor do trimestre.
   
- Fechamento(Close): Utilização do método “last”, atribuindo o último valor do trimestre.
      
- Máximo(High):Utilização do método “max”, atribuindo o máximo valor do trimestre.
 
- Mínimo(Low):Utilização do método “min”, atribuindo o mínimo valor do trimestre.

- Volume(Volume): Utilização do método “mean”, atribuindo a média dos valores do trimestre.

- Fechamento Ajustado(Adj Close):Utilização do método “mean”, atribuindo a média dos valores do trimestre.



Depois, novamente o processo de criar uma coluna "Quartes" com uma identificação mais intuitiva do trimestre e ano, e coloca-lo como a primeira posição das colunas.

Além de renomear a coluna "Month" para "Cluster_Month", tudo no mesmo padrão dos dados fundamentalistas.

In [ ]:
final_data_price = price_data.groupby(['Year','Month']).agg({'High': 'max', 'Low': 'min', 'Open':'first', 'Close' : 'last', 'Volume':'mean', 'Adj Close':'mean'}).reset_index()


final_data_price['Quarter'] = '3Q2012','4Q2012', '1Q2013', '2Q2013', '3Q2013', '4Q2013','1Q2014', '2Q2014', '3Q2014','4Q2014', '1Q2015','2Q2015','3Q2015','4Q2015','1Q2016', '2Q2016', '3Q2016', '4Q2016','1Q2017','2Q2017','3Q2017','4Q2017','1Q2018','2Q2018'


quarter = final_data_price['Quarter']
final_data_price.drop(labels=['Quarter'], axis=1, inplace=True)
final_data_price.insert(0, 'Quarter', quarter)

final_data_price.head()

# UNINDO DADOS DE PREÇO E DADOS FUNDAMENTALISTAS

Simplesmente fazendo um merge entre os dados de preço e fundamentalistas que tratamos até aqui.

A coluna "Year" e "Month" ficará repetida, então precisaremos renomear cada uma e excluir as outras duas que são a mesma coisa.

In [ ]:
dataset = pd.merge(final_data_price, fundamental_data, how='left', on='Quarter')

dataset.rename(columns={'Year_x' : 'Year'},inplace = True)
dataset.rename(columns={'Cluster_Month_x' : 'Cluster_Month'},inplace = True)

dataset.drop('Year_y', axis=1, inplace=True)

dataset.drop('Cluster_Month', axis=1, inplace=True)

dataset.head()

# CRIAÇÃO DO TARGET

## Good: Valorizou mais que 10% em um ano(4 trimestres)
## Bad: Não valorizou mais que 10% em um ano(4 trimestres)

Para criarmos o target, que será a variavel a ser predita nos nossos algoritmos de classificação, utilizamos o mesmo processo que na criação dos dados de preço forncecidos pela API da Yahoo Finance. Só que agora com algumas mudanças

Repare na data "end", ela tem exatamente um ano a mais que a que utilizamos na composição do dataset de preços, pois precisamos dessa janela que ficará "de fora" para compor o nosso target.

Imaginemos o seguinte: Analisando o trimestre atual, com todos os atributos de preço e fundamentalistas, o preço 4 trimestres depois(1 ano) aumentou 10% ?

Com base nessa lógica, precisamos desses dados de fora da janela para compor a coluna de target.

Ao final dessa célula, temos apenas o mesmo processamento dos dados de preço explicados antes, apenas com a diferença que temos 4 linhas de dados a mais, para que depois consigamos preencher o target.

In [ ]:
##############################3COMEÇA O CÓDIGO QUE EXTRAI DADOS DE PREÇO - ALTERAR#################################
start = datetime(2012, 6, 30)
end = datetime(2019, 5, 31)
price_target = web.get_data_yahoo('NATU3.SA', start, end, interval = 'm')

price_target["Date"] = price_target.index
price_target.reset_index(drop=True, inplace=True)


date = price_target['Date']
price_target.drop(labels=['Date'], axis=1,inplace = True)
price_target.insert(0, 'Date', date)

price_target["Day"] = price_target['Date'].map(lambda x: x.day)
price_target["Month"] = price_target['Date'].map(lambda x: x.month)
price_target["Year"] = price_target['Date'].map(lambda x: x.year)

price_target.drop(columns = 'Date', inplace = True)



year = price_target['Year']
price_target.drop(labels=['Year'], axis=1,inplace = True)
price_target.insert(0, 'Year', year)

month = price_target['Month']
price_target.drop(labels=['Month'], axis=1,inplace = True)
price_target.insert(1, 'Month', month)

day = price_target['Day']
price_target.drop(labels=['Day'], axis=1,inplace = True)
price_target.insert(2, 'Day', day)



for i,row in price_target.iterrows():   
    if (row['Month'] == 1) | (row['Month'] == 2) | (row['Month'] == 3):
        price_target.loc[i,'Month'] = 3
    
    elif (row['Month'] == 4) | (row['Month'] == 5) | (row['Month'] == 6):
         price_target.loc[i,'Month'] = 6
        
    elif (row['Month'] == 7) | (row['Month'] == 8) | (row['Month'] == 9):
         price_target.loc[i,'Month'] = 9
        
    else: 
          price_target.loc[i,'Month'] = 12



price_target.drop('Day', axis=1, inplace=True)

final_price_target = price_target.groupby(['Year','Month']).agg({'High': 'max', 'Low': 'min', 'Open':'first', 'Close' : 'last', 'Volume':'mean', 'Adj Close':'mean'}).reset_index()

########################################ALTERAR##################################
final_price_target['Quarter'] =  '3Q2012','4Q2012', '1Q2013', '2Q2013', '3Q2013', '4Q2013','1Q2014', '2Q2014', '3Q2014','4Q2014', '1Q2015','2Q2015','3Q2015','4Q2015','1Q2016', '2Q2016', '3Q2016', '4Q2016','1Q2017','2Q2017','3Q2017','4Q2017','1Q2018','2Q2018','3Q2018','4Q2018','1Q2019','2Q2019'


quarter = final_price_target['Quarter']
final_price_target.drop(labels=['Quarter'], axis=1, inplace=True)
final_price_target.insert(0, 'Quarter', quarter)

final_price_target.rename(columns={'Month' : 'Cluster_Month'},inplace = True)

final_price_target.head()

Com base nesse dataset de preço, feito exclusivamente para preenchimento do target, constuiremos duas colunas, uma somente com os preços e a outra do target que usará essa coluna e a lógica explicada para preencher a classificação como "Good" ou "Bad".

In [ ]:
price = final_price_target

price = price.iloc[0:,6].to_frame()

price["Date"] = price.index
price.reset_index(drop=True, inplace=True)

date = price['Date']
price.drop(labels=['Date'], axis=1,inplace = True)
price['target'] = ""

price = price.iloc[::-1]

price.head()

A logica abaixo percorre todas as linhas para preencher o target conforme as condições estabelecidas. Como os ultimos 4 são de fora da janela, ele são preenchidos com "Nan", serão cortados pois sua utilidade era ser base para preenchimento dos seus 4 anteriores.

In [ ]:
for i,row in price.iterrows():
    if i > 23:
        price.loc[i,'target'] = 'NaN' 
    else:
        if price.loc[i+4,'Close'] >= (price.loc[i,'Close'] + (price.loc[i,'Close'] *0.1)):
            price.loc[i,'target'] = 'Good'
        else:
            price.loc[i,'target'] = 'Bad'

price.head()

# UNIÃO DE TUDO: ÚLTIMA MANIPULAÇÃO

Finalmente, cortamos as linhas que não vamos usar(com 'nan' no target), e invertemos a ordem do dataset do target para coloca-lo em uma coluna "target" no dataset onde estão os dados de preço e fundamentalistas.

E por fim, usamos uma variavel auxiliar e inserimos a coluna target como a coluna na quarta posição do nosso dataset.

In [ ]:
price = price.iloc[4:,:]
price = price.iloc[::-1]
price.reset_index(drop = True, inplace = True)
dataset['target'] = price.iloc[0:,1].to_frame()
data = dataset['target']
dataset.drop(labels=['target'], axis=1,inplace = True)
dataset.insert(3, 'target', data)

dataset.head()

In [ ]:
dataset.info()

In [ ]:
dataset.to_csv("FINAL_CODE_NATU3.csv", index= False)